In [418]:
import numpy as np
import pandas as pd
from numpy import log2 as log
from sklearn.preprocessing import Imputer
eps = np.finfo(float).eps


#print(eps)
dataset = {'Taste':['Salty','Spicy','Spicy','Spicy','Spicy','Sweet','Salty','Sweet','Spicy','Salty'],
       'Temperature':['Hot','Hot','Hot','Cold','Hot','Cold','Cold','Hot','Cold','Hot'],
       'Texture':['Soft','Soft','Hard','Hard','Hard','Soft','Soft','Soft','Soft','Hard'],
       'Eat':['No','No','Yes','No','Yes','Yes','No','Yes','Yes','Yes']}



df = pd.DataFrame(dataset,columns= list(dataset.keys()))
#print(list(dataset.keys()))


def preProcess(df):
    imputer = Imputer(missing_values ='NaN', strategy = 'mean', axis = 0)
    df.drop('customerID', axis = 1, inplace=True)
    df.drop('TotalCharges', axis = 1, inplace=True)
    df['SeniorCitizen'] = df['SeniorCitizen'].astype(object)
    #print(df.head)
    print(df.dtypes)
    
    #imputer = imputer.fit(df)
    #imputer_data = Imputer.transform(df['tenure'].values)
    
df = pd.read_csv('telco.csv')
df.dropna(subset = ['Churn'])

#df = pd.read_csv('creditcard.csv')
preProcess(df)
#print(df.head)
used = {}
uniqueFeaturesLables = {}
for attr in list(df.keys()):
    uniqueFeaturesLables[attr] = []
#print(df['Taste'])


for attr in list(df.keys()):
    for val in df[attr]:
        if(val not in uniqueFeaturesLables[attr]):
            uniqueFeaturesLables[attr].append(val)
'''
for attr in list(df.keys()):
    print(uniqueFeaturesLables[attr])
print(len(uniqueFeaturesLables))
'''
#df = pd.DataFrame(dataset)
print(type(df))
#print(df)

gender               object
SeniorCitizen        object
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
Churn                object
dtype: object
<class 'pandas.core.frame.DataFrame'>


In [392]:
def Entropy(df, attr = None):
    label = df.keys()[-1]
    values = df[label].unique()
    #print(values)
    entropy = 0.0
    
    for value in values:
        fraction = df[label].value_counts()[value]/len(df[label])
        entropy += -fraction*np.log2(fraction)
    if attr == None:
        return (entropy)
    entropy = 0.0
    final_entropy = 0
    attr_variables = df[attr].unique()
    for variable in attr_variables:
        entropy = 0.0
        for value in values:
            num = len(df[attr][df[attr] == variable][df[label] ==value])
            den = len(df[attr][df[attr] == variable])
            fraction = num/(den+eps)
            #print(den)
            entropy += -fraction*log(fraction+eps)
            fraction2 = den/len(df)
            final_entropy += fraction2*entropy
    #print(attr_variables)
    return (final_entropy)

def ContEntropy(df, attr):
    label = df.keys()[-1]
    values = df[label].unique()
    entropy = 0.0
    total_yes = 0
    total_no = 0
    now_yes = 0
    now_no = 0
    total = 0
    for value in values:
        fraction = df[label].value_counts()[value]/len(df[label])
        if value == 'Yes':
            total_yes = df[label].value_counts()[value]
        else:
            total_no = df[label].value_counts()[value]
        entropy += -fraction*np.log2(fraction)
    total = total_yes+total_no
    entropy = 0.0
    final_entropy = 100
    partition_value = 0
    #attr_variables = df[attr].unique()
    attr_variables = []
    for i in range(len(df)):
        attr_variables.append((df[attr][i],df[label][i]))
    attr_variables = sorted(attr_variables,key = lambda element : element[0])
    #print((attr_variables))
    
    for i in range(len(attr_variables)):
        entropy = 0.0
        if(attr_variables[i][1]=='Yes'):
            now_yes += 1
        else:
            now_no += 1
        other_yes = total_yes - now_yes
        other_no = total_no - now_no
        if(i+1 != len(attr_variables) and attr_variables[i][0]==attr_variables[i+1][0]):
            continue
        entropy += -((now_yes+now_no)/(total+eps))*(((now_yes/(now_yes+now_no+eps))*log(now_yes/(now_yes+now_no+eps)))+((now_no/(now_yes+now_no+eps))*log(now_no/(now_yes+now_no+eps))))
        entropy += -((other_yes+other_no)/(total+eps))*(((other_yes/(other_yes+other_no+eps))*log(other_yes/(other_yes+other_no+eps)))+((other_no/(other_yes+other_no+eps))*log(other_no/(other_yes+other_no+eps))))
        if(entropy <= final_entropy):
            partition_value = attr_variables[i][0]
        final_entropy = min(final_entropy, entropy)
    '''
    
    for variable in attr_variables:
        entropy = 0.0
        for value in values:
            num = len(df[attr][df[attr] <= variable][df[label] ==value])
            den = len(df[attr][df[attr] <= variable])
            fraction = num/(den+eps)
            entropy += -fraction*log(fraction+eps)
        if(entropy <= final_entropy):
            partition_value = variable
        final_entropy = min(final_entropy, entropy)
    '''
    return final_entropy, partition_value

def find_winner(df):
    #print(df.keys())
    #Entropy_att = []
    gain = []
    partition = []
    for key in df.keys()[:-1]:
        #print(df.dtypes[key])
        if df.dtypes[(key)] == 'object':
            #print("dhukse")
            gain.append(Entropy(df)-Entropy(df,key))
            partition.append("Null")
        else:
            #print("dhuke nai")
            entrpy , parttn = ContEntropy(df,key)
            gain.append(Entropy(df)-entrpy)
            partition.append(parttn)
    idx =  np.argmax(gain)
    print(partition)
    return df.keys()[:-1][np.argmax(gain)],partition[idx] 
  
def get_subtable(df,index,value):
    return df[df[index] == value].reset_index(drop=True)

def get_subtable_cont(df,index,value):
    less_table = df[df[index] <= value].reset_index(drop=True)
    greater_table = df[df[index] > value].reset_index(drop=True)
    return less_table, greater_table

find_winner(df)

C:\Users\HP\Anaconda3\envs\workshop\lib\site-packages\ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in log2
C:\Users\HP\Anaconda3\envs\workshop\lib\site-packages\ipykernel_launcher.py:65: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\HP\Anaconda3\envs\workshop\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: divide by zero encountered in log2
C:\Users\HP\Anaconda3\envs\workshop\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: invalid value encountered in double_scalars


['Null', 'Null', 'Null', 'Null', 17, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 26.899999999999999]


('tenure', 17)

In [394]:

def makeTree(df,depth, max_depth, tree = None, parent_class = None):
    Class = df.keys()[-1]
    #print(Class)
    #if(len(df[Class].unique())==1):
    #    return str(df[Class].unique()[0])
    #print(Class)
    partition_index, cont_partition_val = find_winner(df)
    #print(partition_index)
    #partition_values= np.unique(df[partition_index])
    partition_values = uniqueFeaturesLables[partition_index]
    if tree == None:
        tree = {}
        tree[partition_index] = {}
    #values = uniqueFeaturesLables[Class]
    values = df[Class].unique()
    #print(values)
    #print(values)
    x = 0
    majority_class = values[0]
    for value in values:
        #print(value)
        y = df[Class].value_counts()[value]
        if(y>x):
            x = y
            majority_class = value
    if depth==len(uniqueFeaturesLables) or depth == max_depth:
        return str(majority_class)
    if cont_partition_val == "Null":
        for value in partition_values:
            table = get_subtable(df, partition_index, value)
            ClassVal,cnt = np.unique(table[Class],return_counts=True)
            #print("ClassVal = "+ ClassVal);
            if len(cnt) == 1:
                tree[partition_index][value] = ClassVal[0]
            elif(len(table) == 0):
                tree[partition_index][value] = str(majority_class)
            else:

                tree[partition_index][value] = makeTree(table, depth+1, max_depth ,parent_class=majority_class)
    else:
        less_table, greater_table = get_subtable_cont(df, partition_index, cont_partition_val)
        less_ClassVal,less_cnt = np.unique(less_table[Class],return_counts=True)
        greater_ClassVal,greater_cnt = np.unique(less_table[Class],return_counts=True)
        done = 0
        #print("ClassVal = "+ ClassVal);
        if(done == 0):
            if len(less_cnt) == 1:
                tree[partition_index]["<="+str(cont_partition_val)] = less_ClassVal[0]
            elif(len(less_table) == 0):
                tree[partition_index]["<=" + str(cont_partition_val)] = str(majority_class)
            else:
                tree[partition_index]["<=" + str(cont_partition_val)] = makeTree(less_table, depth+1, max_depth ,parent_class=majority_class)
        if(done == 0):
            if len(greater_cnt) == 1:
                tree[partition_index][str(cont_partition_val)] = greater_ClassVal[0]
            elif(len(greater_table) == 0):
                tree[partition_index][str(cont_partition_val)] = str(majority_class)
            else:
                tree[partition_index][str(cont_partition_val)] = makeTree(greater_table, depth+1, max_depth ,parent_class=majority_class)
            
    return tree
    
    
tree = makeTree(df, depth = 0, max_depth = 100)

C:\Users\HP\Anaconda3\envs\workshop\lib\site-packages\ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in log2
C:\Users\HP\Anaconda3\envs\workshop\lib\site-packages\ipykernel_launcher.py:65: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\HP\Anaconda3\envs\workshop\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: divide by zero encountered in log2
C:\Users\HP\Anaconda3\envs\workshop\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: invalid value encountered in double_scalars


['Null', 'Null', 'Null', 'Null', 17, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 26.899999999999999]
['Null', 'Null', 'Null', 'Null', 5, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 68.599999999999994]
['Null', 'Null', 'Null', 'Null', 3, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 20.850000000000001]
['Null', 'Null', 'Null', 'Null', 1, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 20.850000000000001]
['Null', 'Null', 'Null', 'Null', 1, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 19.199999999999999]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 19.199999999999999]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Nu

['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 19.899999999999999]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'N

['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Nu

['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Nu

...

['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Nu

['Null', 'Null', 'Null', 'Null', 56, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 94.799999999999997]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 99.900000000000006]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Nu

['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 64, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 107.5]
['Null', 'Null', 'Null', 'Null', 64, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 106.0]
['Null', 'Null', 'Null', 'Null', 61, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 105.40000000000001]
['Null', 'Null', 'Null', 'Null', 62, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 105.5]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null

['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 69, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 115.8]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 117.2]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 0]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'N

{'tenure': {'17': {'MonthlyCharges': {'69.95': {'tenure': {'55': {'tenure': {'70': {'MonthlyCharges': {'93.25': {'MonthlyCharges': {'109.25': {'Contract': {'Month-to-month': 'No',
              'One year': {'Partner': {'No': 'No',
                'Yes': {'OnlineSecurity': {'No': 'No',
                  'No internet service': 'No',
                  'Yes': {'TechSupport': {'No': 'No',
                    'No internet service': 'No',
                    'Yes': 'Yes'}}}}}},
              'Two year': 'No'}},
            '<=109.25': {'MonthlyCharges': {'103.2': {'tenure': {'71': {'MonthlyCharges': {'104.15': {'PaymentMethod': {'Bank transfer (automatic)': 'No',
                    'Credit card (automatic)': 'No',
                    'Electronic check': {'gender': {'Female': {'OnlineSecurity': {'No': 'Yes',
                        'No internet service': 'No',
                        'Yes': 'No'}},
                      'Male': 'No'}},
                    'Mailed check': 'No'}},
             

In [399]:
import pprint
tree = makeTree(df, depth = 0, max_depth = 4)
pprint.pprint(tree)

C:\Users\HP\Anaconda3\envs\workshop\lib\site-packages\ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in log2
C:\Users\HP\Anaconda3\envs\workshop\lib\site-packages\ipykernel_launcher.py:65: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\HP\Anaconda3\envs\workshop\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: divide by zero encountered in log2
C:\Users\HP\Anaconda3\envs\workshop\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: invalid value encountered in double_scalars


['Null', 'Null', 'Null', 'Null', 17, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 26.899999999999999]
['Null', 'Null', 'Null', 'Null', 5, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 68.599999999999994]
['Null', 'Null', 'Null', 'Null', 3, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 20.850000000000001]
['Null', 'Null', 'Null', 'Null', 1, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 20.850000000000001]
['Null', 'Null', 'Null', 'Null', 5, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 20.899999999999999]
['Null', 'Null', 'Null', 'Null', 1, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 84.349999999999994]
['Null', 'Null', 'Null', 'Null', 0, 'Null', 'Null', 'Null', 'Null', 'Null', 'Null', 'Nu

In [423]:
import pprint
#tree = makeTree(df, depth = 0, max_depth = 2)
pprint.pprint(tree)

unique_labels = df[df.keys()[-1]].unique()
unique_labels
print(unique_labels)
def predict(tree, inst):
    #pprint.pprint(tree)
    for nodes in tree.keys():
        if(nodes in unique_labels):
            return nodes
        value = inst[nodes]
        
        if df.dtypes[nodes] != 'object':
            #print(list(tree[nodes].keys())[0])
            
            if list(tree[nodes].keys())[0][0]=='<':
                less_value = list(tree[nodes].keys())[0]
                greater_value = list(tree[nodes].keys())[1]
            else:
                less_value = list(tree[nodes].keys())[1]
                greater_value = list(tree[nodes].keys())[0]
                
                
            
            if((df.dtypes[nodes] == 'int64' and inst[nodes] <= int(less_value[2:])) or (df.dtypes[nodes] == 'float64' and inst[nodes] <= float(less_value[2:]))):
                tree = (tree[nodes][less_value])
            else:
                tree = (tree[nodes][greater_value])
            
        
        #print("nodes = " + nodes)
        #print("value = " + value)
        #tree = (tree[nodes][value])
        
        
        prediction = 'No'
        if type(tree) is dict:
            prediction = predict(tree, inst)
        else:
            prediction = tree
            break;  
    return prediction
    if prediction=='Yes':
        return 1.0
    return -1.0

inst = df.iloc[6]
#print(inst['Taste'])
#predict(tree, inst)
for i in range(10):
    print(predict(tree, df.iloc[i]))

{'tenure': {'17': {'MonthlyCharges': {'69.95': 'No', '<=69.95': 'No'}},
            '<=17': {'MonthlyCharges': {'68.6': 'Yes', '<=68.6': 'No'}}}}
['No' 'Yes']
No
No
No
No
Yes
Yes
No
No
No
No


In [309]:
#returns a vector of trees and weights of trees 
from numpy.random import choice
feature_list = df.keys()

def make_feedable(data):
    data_dic = {}
    for i in feature_list:
        data_dic[i] = []
    for j in feature_list:
        for i in data:
            data_dic[j].append(df[j][i])
    #print(data_dic)
    dic_df = pd.DataFrame(data_dic,columns= feature_list)
    return dic_df


def AdaBoost(df, K = 5):
    data_weights = {}
    Hypothesis_vector = []
    Hypothesis_weights = []
    list_of_candidates = []
    sampled = []
    for i in range(len(df)):
        data_weights[i] = 1.0/len(df)
        sampled.append(i)
        #print(data_weights[i])
    #print(data_weights.values())
    
    for j in range(len(df)):
        list_of_candidates.append(j)
        
    i = 0
    #change it
    new_df = df
    while (i<K):
        tree = makeTree(new_df, 0, 1)
        #print(sampled)
        error = 0.0
        for j in range(len(df)):
            prediction = predict(tree, df.iloc[j])
            if(prediction is not df[df.keys()[-1]][j]):
                error = error + data_weights[j]
                #print(error)
        if error>0.5:
            i = i-1
            #print(i)
            continue
        for j in range(len(df)):
            prediction = predict(tree, df.iloc[j])
            if(prediction is df[df.keys()[-1]][j]):
                data_weights[j] = (data_weights[j]*error)/(1-error)
        #normalization_step
        weight_array = list(data_weights.values())
        #print(type(weight_array[0]))
        weight_array = np.array(weight_array)
        #print(np.sum(weight_array))
        weight_array = weight_array/np.sum(weight_array)
        
        for j in data_weights.keys():
            data_weights[j] = weight_array[j]
        Hypothesis_weights.append(log((1-error)/error))
        Hypothesis_vector.append(tree)
        sampled = choice(list_of_candidates, int(0.4*len(df)), p=list(data_weights.values()))
        new_df = make_feedable(sampled)
        i = i + 1
        
    return Hypothesis_vector, Hypothesis_weights    
Hypothesis_vector, Hypothesis_weights = AdaBoost(df)

In [310]:
def final_prediction(Hypothesis_vector, Hypothesis_weights, instance):
    outpt = 0.0
    for i in range(len(Hypothesis_vector)):
        if(predict(Hypothesis_vector[i], instance) == 'Yes'):
           outpt = outpt + Hypothesis_weights[i]
        else:
           outpt = outpt - Hypothesis_weights[i]
    if outpt>=0:
        print("Yes")
    else:
        print("No")

In [312]:
for i in range(10):
    final_prediction(Hypothesis_vector, Hypothesis_weights, df.iloc[i])

No
No
Yes
No
Yes
Yes
No
Yes
Yes
Yes
